# Query Iceberg Tables with DuckDB

Uses **PyIceberg** (pointed at the Nessie REST catalog) to resolve table metadata paths,
then hands those paths to **DuckDB**'s native `iceberg_scan` for fast SQL queries against MinIO.

| Layer | Tool |
|-------|------|
| Catalog | Nessie REST (`localhost:19120`) |
| Storage | MinIO (`minio-api.andylile.com`) |
| Query engine | DuckDB + `iceberg` + `httpfs` extensions |

In [75]:
%pip install duckdb pyiceberg s3fs pandas --quiet

Note: you may need to restart the kernel to use updated packages.


## 1 — Configuration

Edit the values below to match your environment. `MINIO_ACCESS_KEY` and `MINIO_SECRET_KEY`
are the same credentials used in `.env`.

In [76]:
# ── Nessie ────────────────────────────────────────────────────────────────────
NESSIE_URI      = "http://localhost:19120/iceberg"   # exposed port in docker-compose.override.yml

# ── MinIO ─────────────────────────────────────────────────────────────────────
MINIO_ENDPOINT  = "https://minio-api.andylile.com"
MINIO_ACCESS_KEY = "LwL2a0Lic5JYaXvxl06X"
MINIO_SECRET_KEY = "v60a4nh1nebmrbFbwoS6AFcaCQYd3G5Gu11qH5Kf"

# ── Iceberg warehouse / namespace ─────────────────────────────────────────────
WAREHOUSE       = "themeparks"    # matches nessie.catalog.default-warehouse
NAMESPACE       = "silver"        # actual namespace used by the DAGs


## 2 — Connect to Nessie catalog via PyIceberg

In [77]:
from pyiceberg.catalog import load_catalog

catalog = load_catalog(
    "nessie",
    **{
        "type": "rest",
        "uri": NESSIE_URI,
        "s3.endpoint": MINIO_ENDPOINT,
        "s3.access-key-id": MINIO_ACCESS_KEY,
        "s3.secret-access-key": MINIO_SECRET_KEY,
        "s3.path-style-access": "true",
    },
)

print("Namespaces:", catalog.list_namespaces())

Namespaces: [('silver',)]


In [78]:
# List tables in the namespace
tables = catalog.list_tables(NAMESPACE)
print(f"Tables in '{NAMESPACE}':")
for t in tables:
    print(" ", t)

Tables in 'silver':
  ('silver', 'destinations')
  ('silver', 'entities')
  ('silver', 'live_data')
  ('silver', 'parks')


In [112]:
# Diagnostics — list everything Nessie knows about
print("=== Namespaces ===")
namespaces = catalog.list_namespaces()
if not namespaces:
    print("  (none — Nessie catalog is empty)")
for ns in namespaces:
    print(f"  {ns}")
    tables = catalog.list_tables(ns)
    if tables:
        for t in tables:
            print(f"    └─ {t[1]}")
    else:
        print("      (no tables)")


=== Namespaces ===
  ('gold',)
    └─ live_data_current
  ('silver',)
    └─ destinations
    └─ entities
    └─ live_data
    └─ parks


## 3 — Set up DuckDB with S3 / Iceberg support

In [80]:
import duckdb

con = duckdb.connect()

# Install and load required extensions
con.execute("INSTALL httpfs;   LOAD httpfs;")
con.execute("INSTALL iceberg;  LOAD iceberg;")

# Point DuckDB at MinIO
con.execute(f"""
    SET s3_endpoint    = '{MINIO_ENDPOINT.replace('https://', '').replace('http://', '')}';
    SET s3_access_key_id     = '{MINIO_ACCESS_KEY}';
    SET s3_secret_access_key = '{MINIO_SECRET_KEY}';
    SET s3_use_ssl     = true;
    SET s3_url_style   = 'path';
    SET s3_region      = 'us-east-1';
""")

print("DuckDB ready")

DuckDB ready


## 4 — Helper: resolve Iceberg metadata path from Nessie

In [81]:
def metadata_location(namespace: str, table_name: str) -> str:
    """Return the s3:// path to the current Iceberg metadata JSON file."""
    tbl = catalog.load_table(f"{namespace}.{table_name}")
    return tbl.metadata_location

def query_table(namespace: str, table_name: str, sql_where: str = "") -> duckdb.DuckDBPyRelation:
    """Query an Iceberg table using DuckDB's iceberg_scan."""
    path = metadata_location(namespace, table_name)
    where = f"WHERE {sql_where}" if sql_where else ""
    return con.execute(f"SELECT * FROM iceberg_scan('{path}') {where}").df()

print("Helper functions defined")

Helper functions defined


## 5 — Query the tables

### Destinations

In [82]:
df_destinations = query_table(NAMESPACE, "destinations")
print(f"{len(df_destinations)} rows")
df_destinations.head(10)

96 rows


,id,name,slug,parks,ingest_timestamp
0,259cf011-6195-42dd-bfdb-640969e0bfb9,Guangzhou Chimelong Tourist Resort,chimelongguangzhou,[{'id': '73436fe5-1f14-400f-bfbf-ab6766269e70'...,2026-02-20T14:00:00.647196+00:00
1,9c6a0987-e519-4d6e-b011-e6c47a60641b,Walibi Holland,walibiholland,[{'id': '18635b3e-fa23-4284-89dd-9fcd0aaa9c9c'...,2026-02-20T14:00:00.647196+00:00
2,6cc48df2-f126-4f28-905d-b4c2c15765f2,Parc Asterix,parcasterix,[{'id': '9e938687-fd99-46f3-986a-1878210378f8'...,2026-02-20T14:00:00.647196+00:00
3,c0eddd5b-da82-4161-9a5f-2eb4ab5f82e7,Plopsaland Belgium,plopsaland-de-panne,[{'id': 'f0ea9b9c-1ccb-4860-bfe6-b5aea7e4db2b'...,2026-02-20T14:00:00.647196+00:00
4,8d8a8cc7-4523-4437-8bb6-5c87a26ba5ce,Walibi Belgium,walibibelgium,[{'id': '21897ba2-cc63-460e-99b0-fcd4ca7c18ef'...,2026-02-20T14:00:00.647196+00:00
5,8fba5a14-8d04-455c-acf8-eccaaa0f58d9,Silver Dollar City,silverdollarcity,[{'id': 'd21fac4f-1099-4461-849c-0f8e0d6e85a6'...,2026-02-20T14:00:00.647196+00:00
6,f9497403-adf3-4409-bd79-bb5b54000e45,Walibi Rhône-Alpes,walibirhonealpes,[{'id': '28aee1df-1d05-4f53-bbf5-08f7aabff3a1'...,2026-02-20T14:00:00.647196+00:00
7,6c3cd0cc-57b5-431b-926c-2658e8104057,Dollywood,dollywood,[{'id': '7502308a-de08-41a3-b997-961f8275ab3c'...,2026-02-20T14:00:00.647196+00:00
8,0257ff9f-c73c-4855-b5b4-774755c4d146,Phantasialand,phantasialand,[{'id': 'abb67808-61e3-49ef-996c-1b97ed64fac6'...,2026-02-20T14:00:00.647196+00:00
9,ae0cd07c-87f3-41d9-a825-0fded65d626c,LEGOLAND Japan,legolandjapanresort,[{'id': '0c14c187-13f5-41ef-91b1-7ecb504239a7'...,2026-02-20T14:00:00.647196+00:00


### Entities

In [83]:
df_entities = query_table(NAMESPACE, "entities")
print(f"{len(df_entities)} rows")
df_entities.head(10)

7219 rows


,park_id,id,name,entityType,ingest_timestamp
0,73436fe5-1f14-400f-bfbf-ab6766269e70,9aa0bc92-6b9c-4bf9-9319-810c9f9a2e0f,火箭过山车,ATTRACTION,2026-02-20T05:18:43.563571+00:00
1,73436fe5-1f14-400f-bfbf-ab6766269e70,7fb74359-4657-45cf-8b83-6f63b5090227,龙卷风暴,ATTRACTION,2026-02-20T05:18:43.563571+00:00
2,73436fe5-1f14-400f-bfbf-ab6766269e70,24a21c03-0004-4871-9e31-7cfce66e1fde,桑巴气球,ATTRACTION,2026-02-20T05:18:43.563571+00:00
3,73436fe5-1f14-400f-bfbf-ab6766269e70,0a06581f-6862-4e1b-bf33-467b09da168e,极速跳跃,ATTRACTION,2026-02-20T05:18:43.563571+00:00
4,73436fe5-1f14-400f-bfbf-ab6766269e70,15573d5d-8434-4586-a5ec-fbe59c7ac730,飞马家庭过山车,ATTRACTION,2026-02-20T05:18:43.563571+00:00
5,73436fe5-1f14-400f-bfbf-ab6766269e70,7a4b771b-a567-4067-9864-cc3b868db4b2,急流勇进,ATTRACTION,2026-02-20T05:18:43.563571+00:00
6,73436fe5-1f14-400f-bfbf-ab6766269e70,bbcd36d0-e47c-4499-8995-d4238c17f067,梦回兰若,ATTRACTION,2026-02-20T05:18:43.563571+00:00
7,73436fe5-1f14-400f-bfbf-ab6766269e70,e20f7a19-8a77-445f-8222-16e29c5b87bf,超级大摆锤,ATTRACTION,2026-02-20T05:18:43.563571+00:00
8,73436fe5-1f14-400f-bfbf-ab6766269e70,d10b1004-9aa7-44ec-9f12-6a522d8b9311,摇摆屋,ATTRACTION,2026-02-20T05:18:43.563571+00:00
9,73436fe5-1f14-400f-bfbf-ab6766269e70,84de15bc-b573-4825-8385-e590d82c7339,滑翔飞翼,ATTRACTION,2026-02-20T05:18:43.563571+00:00


### Live data

In [84]:
df_live = query_table(NAMESPACE, "live_data")
print(f"{len(df_live)} rows")
df_live.head(10)

8822 rows


,park_id,id,name,entityType,status,lastUpdated,ingest_timestamp,queue_standby_wait,queue_single_rider_wait,queue_paid_standby_wait,queue_return_time_state,queue_return_time_start,queue_return_time_end,queue_boarding_group_wait,queue_boarding_group_start,queue_boarding_group_end,queue_boarding_group_status,queue_boarding_group_next_alloc
0,000c724a-cd0f-41a1-b355-f764902c2b55,96458868-f815-4b8f-b7cb-2b04abda284c,Tornado,ATTRACTION,CLOSED,2025-09-07 16:01:36,2026-02-20T14:16:30.032554+00:00,<NA>,<NA>,<NA>,NaN,NaT,None,None,None,None,NaN,None
1,000c724a-cd0f-41a1-b355-f764902c2b55,50b91bb1-3339-417b-bca8-b3a0c88378e8,Dare Devil Dive,ATTRACTION,OPERATING,2025-10-26 20:29:56,2026-02-20T14:16:30.032554+00:00,0,<NA>,<NA>,NaN,NaT,None,None,None,None,NaN,None
2,000c724a-cd0f-41a1-b355-f764902c2b55,e7245ad4-8c51-4d72-98f8-56fb761a6e36,Shipwreck Cove,ATTRACTION,CLOSED,2025-08-24 22:49:24,2026-02-20T14:16:30.032554+00:00,<NA>,<NA>,<NA>,NaN,NaT,None,None,None,None,NaN,None
3,000c724a-cd0f-41a1-b355-f764902c2b55,43ab7687-0092-4b64-b465-1f607e331791,Bucket Blasters,ATTRACTION,CLOSED,2025-08-24 22:49:23,2026-02-20T14:16:30.032554+00:00,<NA>,<NA>,<NA>,NaN,NaT,None,None,None,None,NaN,None
4,000c724a-cd0f-41a1-b355-f764902c2b55,55cbb5ac-cfe7-47b0-9dee-68c6a76dfe08,Typhoon Twister,ATTRACTION,CLOSED,2025-08-31 00:16:23,2026-02-20T14:16:30.032554+00:00,<NA>,<NA>,<NA>,NaN,NaT,None,None,None,None,NaN,None
5,000c724a-cd0f-41a1-b355-f764902c2b55,79610610-c5ce-4f3c-a7e8-fe5bb82eb604,Comet,ATTRACTION,OPERATING,2025-10-12 23:17:34,2026-02-20T14:16:30.032554+00:00,0,<NA>,<NA>,NaN,NaT,None,None,None,None,NaN,None
6,000c724a-cd0f-41a1-b355-f764902c2b55,094776e3-4ba3-4883-811c-48aa0ecb8319,Adventure River,ATTRACTION,CLOSED,2025-09-07 16:01:35,2026-02-20T14:16:30.032554+00:00,<NA>,<NA>,<NA>,NaN,NaT,None,None,None,None,NaN,None
7,000c724a-cd0f-41a1-b355-f764902c2b55,4fdd1128-b655-40b2-b23c-8e43d1e582b5,Adirondack Outlaw,ATTRACTION,OPERATING,2025-10-27 00:46:24,2026-02-20T14:16:30.032554+00:00,0,<NA>,<NA>,NaN,NaT,None,None,None,None,NaN,None
8,000c724a-cd0f-41a1-b355-f764902c2b55,12c49ef0-8846-46fd-860b-1fbee99eacb0,Rocky’s Ranger Planes,ATTRACTION,CLOSED,2025-06-05 14:03:08,2026-02-20T14:16:30.032554+00:00,<NA>,<NA>,<NA>,NaN,NaT,None,None,None,None,NaN,None
9,000c724a-cd0f-41a1-b355-f764902c2b55,88416604-0318-4201-a39d-8d4adb8df692,Sasquatch Launch,ATTRACTION,OPERATING,2025-09-01 14:34:15,2026-02-20T14:16:30.032554+00:00,0,<NA>,<NA>,NaN,NaT,None,None,None,None,NaN,None


## 6 — Ad-hoc SQL

Run arbitrary SQL against any Iceberg table.

In [85]:
meta_path = metadata_location(NAMESPACE, "live_data")

con.execute(f"""
    SELECT status, COUNT(*) AS cnt
    FROM iceberg_scan('{meta_path}')
    GROUP BY status
    ORDER BY cnt DESC
""").df()

,status,cnt
0,CLOSED,5840
1,OPERATING,2740
2,DOWN,186
3,REFURBISHMENT,56


In [86]:
live_data = metadata_location(NAMESPACE, "live_data")
destinations = metadata_location(NAMESPACE, "destinations")
entities = metadata_location(NAMESPACE, "entities")
parks = metadata_location(NAMESPACE, "parks")

### Schema — live_data

`DESCRIBE` returns each column's name, type, nullability, and key info.
Useful after a schema migration to confirm the flat `queue_*` columns are in place.


In [87]:
con.execute(f"""
    SELECT d.slug, d.*
    FROM iceberg_scan('{destinations}') as d
    WHERE slug = 'waltdisneyworldresort'
""").df()

,slug,id,name,slug_1,parks,ingest_timestamp
0,waltdisneyworldresort,e957da41-3552-4cf6-b636-5babc5cbc4e5,Walt Disney World® Resort,waltdisneyworldresort,[{'id': '75ea578a-adc8-4116-a54d-dccb60765ef9'...,2026-02-20T14:00:00.647196+00:00


In [93]:
con.execute(f"""
    DESCRIBE SELECT * FROM iceberg_scan('{live_data}')
""").df()


,column_name,column_type,null,key,default,extra
0,park_id,VARCHAR,YES,None,None,None
1,id,VARCHAR,YES,None,None,None
2,name,VARCHAR,YES,None,None,None
3,entityType,VARCHAR,YES,None,None,None
4,status,VARCHAR,YES,None,None,None
5,lastUpdated,TIMESTAMP,YES,None,None,None
6,ingest_timestamp,VARCHAR,YES,None,None,None
7,queue_standby_wait,BIGINT,YES,None,None,None
8,queue_single_rider_wait,BIGINT,YES,None,None,None
9,queue_paid_standby_wait,BIGINT,YES,None,None,None


In [110]:
# ── Strategy A: Filter FIRST, then deduplicate ───────────────────────────────
# WHERE runs before the window function.
# Only EPCOT attractions enter the ROW_NUMBER() sort.
# Best for a one-off query — minimises the rows the expensive window sort sees.
print("=== Strategy A: filter first, deduplicate second ===")
print(con.execute(f"""
    EXPLAIN
    WITH ranked AS (
        SELECT
            ld.*,
            ROW_NUMBER() OVER (
                PARTITION BY ld.id
                ORDER BY ld.ingest_timestamp DESC
            ) AS rn
        FROM iceberg_scan('{live_data}') ld
            JOIN iceberg_scan('{parks}') p ON ld.park_id = p.id
        WHERE ld.entityType = 'ATTRACTION'
          AND p.name = 'EPCOT'
    )
    SELECT * EXCLUDE (rn) FROM ranked WHERE rn = 1
    ORDER BY queue_standby_wait DESC NULLS LAST
""").fetchall()[0][1])


=== Strategy A: filter first, deduplicate second ===
┌───────────────────────────┐
│          ORDER_BY         │
│    ────────────────────   │
│ ranked.queue_standby_wait │
│            DESC           │
└─────────────┬─────────────┘
┌─────────────┴─────────────┐
│         PROJECTION        │
│    ────────────────────   │
│          park_id          │
│             id            │
│            name           │
│         entityType        │
│           status          │
│        lastUpdated        │
│      ingest_timestamp     │
│     queue_standby_wait    │
│  queue_single_rider_wait  │
│  queue_paid_standby_wait  │
│  queue_return_time_state  │
│  queue_return_time_start  │
│   queue_return_time_end   │
│ queue_boarding_group_wait │
│ queue_boarding_group_start│
│  queue_boarding_group_end │
│queue_boarding_group_status│
│queue_boarding_group_next_a│
│            lloc           │
│                           │
│          ~69 rows         │
└─────────────┬─────────────┘
┌─────────────┴──

In [111]:
# ── Strategy B: Deduplicate FIRST, then filter ───────────────────────────────
# ROW_NUMBER() runs across all rows in live_data (all parks, all entity types,
# all historical snapshots). Filter applies after.
# Better if you query the "current state" view many times — materialise it once.
print("=== Strategy B: deduplicate first, filter second ===")
print(con.execute(f"""
    EXPLAIN
    WITH latest AS (
        SELECT
            *,
            ROW_NUMBER() OVER (
                PARTITION BY id
                ORDER BY ingest_timestamp DESC
            ) AS rn
        FROM iceberg_scan('{live_data}')
    )
    SELECT latest.* EXCLUDE (rn)
    FROM latest
        JOIN iceberg_scan('{parks}') p ON latest.park_id = p.id
    WHERE latest.rn = 1
      AND latest.entityType = 'ATTRACTION'
      AND p.name = 'EPCOT'
    ORDER BY queue_standby_wait DESC NULLS LAST
""").fetchall()[0][1])


=== Strategy B: deduplicate first, filter second ===
┌───────────────────────────┐
│          ORDER_BY         │
│    ────────────────────   │
│ latest.queue_standby_wait │
│            DESC           │
└─────────────┬─────────────┘
┌─────────────┴─────────────┐
│         PROJECTION        │
│    ────────────────────   │
│          park_id          │
│             id            │
│            name           │
│         entityType        │
│           status          │
│        lastUpdated        │
│      ingest_timestamp     │
│     queue_standby_wait    │
│  queue_single_rider_wait  │
│  queue_paid_standby_wait  │
│  queue_return_time_state  │
│  queue_return_time_start  │
│   queue_return_time_end   │
│ queue_boarding_group_wait │
│ queue_boarding_group_start│
│  queue_boarding_group_end │
│queue_boarding_group_status│
│queue_boarding_group_next_a│
│            lloc           │
│                           │
│         ~349 rows         │
└─────────────┬─────────────┘
┌─────────────┴──

### Which strategy is optimal?

**Strategy A wins for a single ad-hoc query.** Here's why:

The window function (`ROW_NUMBER() OVER (PARTITION BY id ORDER BY ...)`) is an **O(n log n) sort** — the most expensive operation in either plan. The number of rows it has to sort is the critical variable.

| | Strategy A (filter → deduplicate) | Strategy B (deduplicate → filter) |
|---|---|---|
| Rows entering window sort | ~EPCOT attractions × snapshots | **all** live_data rows × snapshots |
| Join cost | Small (EPCOT parks only) | Full live_data × parks |
| Best when | One-off query | Pre-materialised "current state" view queried repeatedly |

**Your intuition is right in a different context**: if you were building a *gold layer table* (e.g. `gold.live_data_current`) that pre-deduplicates on every pipeline run, Strategy B's layout makes sense — deduplicate once, store the result, and downstream queries filter cheaply against a small table with no window function needed.

For Iceberg + DuckDB scanning Parquet files directly, there is no index or partition pruning on `entityType` or `park_id` at the file level (our tables are unpartitioned), so both strategies do a full Parquet scan. **The only real cost difference is how many rows enter the in-memory window sort.**


In [ ]:
# Strategy A — actual results
con.execute(f"""
    WITH ranked AS (
        SELECT
            ld.*,
            ROW_NUMBER() OVER (
                PARTITION BY ld.id
                ORDER BY ld.ingest_timestamp DESC
            ) AS rn
        FROM iceberg_scan('{live_data}') ld
            JOIN iceberg_scan('{parks}') p ON ld.park_id = p.id
        WHERE ld.entityType = 'ATTRACTION'
          AND p.name = 'EPCOT'
    )
    SELECT * EXCLUDE (rn)
    FROM ranked
    WHERE rn = 1
    ORDER BY queue_standby_wait DESC NULLS LAST
""").df()


In [114]:
con.execute(f"""
    SELECT
        p.destination_name   AS dest_name,
        p.name               AS park_name,
        ld.name              AS entity_name,
        ld.entityType        AS entity_type,
        ld.status,
        ld.queue_standby_wait,
        ld.queue_single_rider_wait,
        ld.queue_paid_standby_wait,
        ld.queue_return_time_state,
        ld.queue_return_time_start,
        ld.queue_return_time_end,
        ld.lastUpdated,
        p.id
    FROM iceberg_scan('{destinations}') AS d
        JOIN iceberg_scan('{parks}')     AS p  ON d.id = p.destination_id
        JOIN iceberg_scan('{live_data}') AS ld ON p.id = ld.park_id
    WHERE d.slug = 'waltdisneyworldresort'
      AND p.name = 'EPCOT'
    ORDER BY ld.queue_standby_wait DESC NULLS LAST
""").df()


,dest_name,park_name,entity_name,entity_type,status,queue_standby_wait,queue_single_rider_wait,queue_paid_standby_wait,queue_return_time_state,queue_return_time_start,queue_return_time_end,lastUpdated,id
0,Walt Disney World® Resort,EPCOT,Guardians of the Galaxy: Cosmic Rewind,ATTRACTION,OPERATING,105,<NA>,<NA>,NaN,NaT,None,2026-02-20 14:11:43,47f90d2c-e191-4239-a466-5892ef59a88b
1,Walt Disney World® Resort,EPCOT,Guardians of the Galaxy: Cosmic Rewind,ATTRACTION,OPERATING,105,<NA>,<NA>,NaN,NaT,None,2026-02-20 14:11:43,47f90d2c-e191-4239-a466-5892ef59a88b
2,Walt Disney World® Resort,EPCOT,Test Track,ATTRACTION,OPERATING,95,<NA>,<NA>,FINISHED,NaT,None,2026-02-20 14:11:59,47f90d2c-e191-4239-a466-5892ef59a88b
3,Walt Disney World® Resort,EPCOT,Test Track,ATTRACTION,OPERATING,95,<NA>,<NA>,FINISHED,NaT,None,2026-02-20 14:11:59,47f90d2c-e191-4239-a466-5892ef59a88b
4,Walt Disney World® Resort,EPCOT,Remy's Ratatouille Adventure,ATTRACTION,OPERATING,70,<NA>,<NA>,AVAILABLE,2026-02-21 00:30:00,None,2026-02-20 14:11:42,47f90d2c-e191-4239-a466-5892ef59a88b
...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,Walt Disney World® Resort,EPCOT,Kidcot Fun Stops,ATTRACTION,OPERATING,<NA>,<NA>,<NA>,NaN,NaT,None,2026-02-20 05:21:47,47f90d2c-e191-4239-a466-5892ef59a88b
130,Walt Disney World® Resort,EPCOT,Biergarten Restaurant,RESTAURANT,OPERATING,<NA>,<NA>,<NA>,NaN,NaT,None,2026-02-20 00:50:27,47f90d2c-e191-4239-a466-5892ef59a88b
131,Walt Disney World® Resort,EPCOT,Marimba de las Américas,SHOW,OPERATING,<NA>,<NA>,<NA>,NaN,NaT,None,2026-02-20 05:36:45,47f90d2c-e191-4239-a466-5892ef59a88b
132,Walt Disney World® Resort,EPCOT,Le Cellier Steakhouse,RESTAURANT,OPERATING,<NA>,<NA>,<NA>,NaN,NaT,None,2026-02-19 21:19:37,47f90d2c-e191-4239-a466-5892ef59a88b


In [ ]:
# Boarding group status across all parks (latest snapshot per entity)
con.execute(f"""
    WITH latest AS (
        SELECT *,
               ROW_NUMBER() OVER (PARTITION BY id ORDER BY ingest_timestamp DESC) AS rn
        FROM iceberg_scan('{live_data}')
        WHERE queue_boarding_group_status IS NOT NULL
    )
    SELECT
        park_id,
        name,
        status,
        queue_boarding_group_status   AS bg_status,
        queue_boarding_group_wait     AS bg_wait,
        queue_boarding_group_start    AS bg_start,
        queue_boarding_group_end      AS bg_end,
        queue_boarding_group_next_alloc AS next_alloc,
        lastUpdated
    FROM latest
    WHERE rn = 1
    ORDER BY park_id, name
""").df()


---

## 7 — Gold layer: `gold.live_data_current`

Pre-deduplicated current state of every entity across all parks. Written by the
`gold_live_data_current` DAG triggered after each silver `iceberg_live_data` run.

Queries here need no window function — one row per entity, always current.


In [115]:
live_data_current = metadata_location("gold", "live_data_current")

df_current = con.execute(f"SELECT * FROM iceberg_scan('{live_data_current}') where park_id = '47f90d2c-e191-4239-a466-5892ef59a88b'").df()
print(f"{len(df_current)} entities (current state)")
df_current.head(10)


67 entities (current state)


,park_id,id,name,entityType,status,lastUpdated,ingest_timestamp,queue_standby_wait,queue_single_rider_wait,queue_paid_standby_wait,queue_return_time_state,queue_return_time_start,queue_return_time_end,queue_boarding_group_wait,queue_boarding_group_start,queue_boarding_group_end,queue_boarding_group_status,queue_boarding_group_next_alloc
0,47f90d2c-e191-4239-a466-5892ef59a88b,1e735ffb-4868-47f1-b2cd-2ac1156cd5f0,Remy's Ratatouille Adventure,ATTRACTION,OPERATING,2026-02-20 15:33:03,2026-02-20T15:35:00.677195+00:00,60.0,NaN,NaN,FINISHED,NaT,None,None,None,None,None,None
1,47f90d2c-e191-4239-a466-5892ef59a88b,61fb49f8-e62f-4e1c-ae0e-8ab9929037bc,Canada Far and Wide in Circle-Vision 360,ATTRACTION,OPERATING,2026-02-20 14:11:41,2026-02-20T15:35:00.677195+00:00,NaN,NaN,NaN,NaN,NaT,None,None,None,None,None,None
2,47f90d2c-e191-4239-a466-5892ef59a88b,3206e3a6-cbc3-4960-9700-163764bc47d6,Le Cellier Steakhouse,RESTAURANT,OPERATING,2026-02-19 21:19:37,2026-02-20T15:35:00.677195+00:00,NaN,NaN,NaN,NaN,NaT,None,None,None,None,None,None
3,47f90d2c-e191-4239-a466-5892ef59a88b,76eadac1-cdfa-45e5-a25c-54ed4c3a604f,Marimba de las Américas,SHOW,OPERATING,2026-02-20 05:36:45,2026-02-20T15:35:00.677195+00:00,NaN,NaN,NaN,NaN,NaT,None,None,None,None,None,None
4,47f90d2c-e191-4239-a466-5892ef59a88b,c9e1d1f6-021f-43f2-a14b-3e67f65adbc4,Biergarten Restaurant,RESTAURANT,OPERATING,2026-02-20 00:50:27,2026-02-20T15:35:00.677195+00:00,NaN,NaN,NaN,NaN,NaT,None,None,None,None,None,None
5,47f90d2c-e191-4239-a466-5892ef59a88b,33bd3bad-6803-4c5e-97ac-f7e31261a604,Meet Anna and Elsa at Royal Sommerhus,SHOW,OPERATING,2026-02-20 14:45:00,2026-02-20T15:35:00.677195+00:00,75.0,NaN,NaN,NaN,NaT,None,None,None,None,None,None
6,47f90d2c-e191-4239-a466-5892ef59a88b,3ace01d1-15fc-4fbb-99e4-81a696cb2d05,Kidcot Fun Stops,ATTRACTION,OPERATING,2026-02-20 05:21:47,2026-02-20T15:35:00.677195+00:00,NaN,NaN,NaN,NaN,NaT,None,None,None,None,None,None
7,47f90d2c-e191-4239-a466-5892ef59a88b,0f40274d-420a-425a-9377-29fd6e49484f,House of the Whispering Willows,ATTRACTION,OPERATING,2026-02-20 05:21:46,2026-02-20T15:35:00.677195+00:00,NaN,NaN,NaN,NaN,NaT,None,None,None,None,None,None
8,47f90d2c-e191-4239-a466-5892ef59a88b,57acb522-a6fc-4aa4-a80e-21f21f317250,Turtle Talk With Crush,ATTRACTION,OPERATING,2026-02-20 15:31:44,2026-02-20T15:35:00.677195+00:00,15.0,NaN,NaN,AVAILABLE,2026-02-20 16:30:00,None,None,None,None,None,None
9,47f90d2c-e191-4239-a466-5892ef59a88b,99087d9a-4380-4ab6-b088-20816e233c6f,Meet Asha Near World Showcase Plaza,SHOW,OPERATING,2026-02-20 05:36:46,2026-02-20T15:35:00.677195+00:00,NaN,NaN,NaN,NaN,NaT,None,None,None,None,None,None


In [ ]:
# EPCOT attractions — no window function needed, gold is already deduplicated
con.execute(f"""
    SELECT
        ld.name,
        ld.status,
        ld.queue_standby_wait,
        ld.queue_single_rider_wait,
        ld.queue_return_time_state,
        ld.lastUpdated
    FROM iceberg_scan('{live_data_current}') ld
        JOIN iceberg_scan('{parks}') p ON ld.park_id = p.id
    WHERE ld.entityType = 'ATTRACTION'
      AND p.name = 'EPCOT'
    ORDER BY ld.queue_standby_wait DESC NULLS LAST
""").df()
